In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, re
from tensorflow import keras
import keras
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn import metrics, preprocessing
from keras.utils.np_utils import to_categorical
from keras.layers import BatchNormalization

In [2]:
train_df = pd.read_csv('large.csv', sep='\t')
test_df = pd.read_csv('test.csv', sep='\t')

In [3]:
train_df = train_df.round(4)
test_df = test_df.round(4)

In [4]:
X = train_df[['peak_1', 'h_1', 'k_1', 'l_1', 'peak_2', 'h_2', 'k_2', 'l_2', 'peak_3', 'h_3', 'k_3', 'l_3', 'peak_4', 'h_4', 'k_4', 'l_4', 'peak_5', 'h_5', 'k_5', 'l_5', 'peak_6', 'h_6', 'k_6', 'l_6']]
y = train_df['space_code']

In [5]:
skf = StratifiedKFold(n_splits=10, shuffle=True)

In [6]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=np.random.seed(42))
cvscores = []
for train, test in kfold.split(X, y):
    
    one_hot_train_labels = to_categorical(y[train])
    one_hot_test_labels = to_categorical(y[test])
    
    X_val = X.iloc[train][:1000]
    partial_X_train = X.iloc[train][1000:]
    y_val = one_hot_train_labels[:1000]
    partial_y_train = one_hot_train_labels[1000:]
    
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_dim=24))
    BatchNormalization()
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(9, activation='softmax'))
    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(partial_X_train, partial_y_train, epochs=500, batch_size=100, validation_data=(X_val, y_val))
    
    scores = model.evaluate(X.iloc[test], one_hot_test_labels, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

Epoch 1/500
46/46 [==============================] - 4s 46ms/step - loss: 118.5501 - accuracy: 0.5526 - val_loss: 7.5053 - val_accuracy: 0.0330
Epoch 2/500
46/46 [==============================] - 1s 15ms/step - loss: 0.7337 - accuracy: 0.7700 - val_loss: 9.4952 - val_accuracy: 0.0360
Epoch 3/500
46/46 [==============================] - 1s 17ms/step - loss: 0.6000 - accuracy: 0.8083 - val_loss: 10.7405 - val_accuracy: 0.0340
Epoch 4/500
46/46 [==============================] - 1s 15ms/step - loss: 219.9810 - accuracy: 0.8282 - val_loss: 13.7772 - val_accuracy: 0.0370
Epoch 5/500
46/46 [==============================] - 1s 15ms/step - loss: 0.5405 - accuracy: 0.8255 - val_loss: 15.4653 - val_accuracy: 0.0360
Epoch 6/500
46/46 [==============================] - 1s 16ms/step - loss: 0.5039 - accuracy: 0.8350 - val_loss: 17.7659 - val_accuracy: 0.0350
Epoch 7/500
46/46 [==============================] - 1s 17ms/step - loss: 0.4481 - accuracy: 0.8487 - val_loss: 19.4027 - val_accuracy: 0.03

Epoch 58/500
46/46 [==============================] - 1s 14ms/step - loss: 0.1290 - accuracy: 0.9567 - val_loss: 40.4751 - val_accuracy: 0.0360
Epoch 59/500
46/46 [==============================] - 1s 15ms/step - loss: 0.1436 - accuracy: 0.9556 - val_loss: 39.5392 - val_accuracy: 0.0360
Epoch 60/500
46/46 [==============================] - 1s 15ms/step - loss: 0.1252 - accuracy: 0.9581 - val_loss: 40.6925 - val_accuracy: 0.0370
Epoch 61/500
46/46 [==============================] - 1s 14ms/step - loss: 0.1223 - accuracy: 0.9631 - val_loss: 41.1333 - val_accuracy: 0.0360
Epoch 62/500
46/46 [==============================] - 1s 15ms/step - loss: 0.1382 - accuracy: 0.9586 - val_loss: 40.0290 - val_accuracy: 0.0360
Epoch 63/500
46/46 [==============================] - 1s 16ms/step - loss: 0.1306 - accuracy: 0.9566 - val_loss: 39.2634 - val_accuracy: 0.0360
Epoch 64/500
46/46 [==============================] - 1s 13ms/step - loss: 0.1224 - accuracy: 0.9619 - val_loss: 40.2875 - val_accuracy:

46/46 [==============================] - 1s 16ms/step - loss: 0.1098 - accuracy: 0.9691 - val_loss: 51.1256 - val_accuracy: 0.0360
Epoch 115/500
46/46 [==============================] - 1s 15ms/step - loss: 0.1068 - accuracy: 0.9763 - val_loss: 49.5293 - val_accuracy: 0.0360
Epoch 116/500
46/46 [==============================] - 1s 15ms/step - loss: 0.1196 - accuracy: 0.9713 - val_loss: 52.4351 - val_accuracy: 0.0370
Epoch 117/500
46/46 [==============================] - 1s 15ms/step - loss: 0.1004 - accuracy: 0.9722 - val_loss: 51.1261 - val_accuracy: 0.0360
Epoch 118/500
46/46 [==============================] - 1s 15ms/step - loss: 8573.4590 - accuracy: 0.9735 - val_loss: 51.5944 - val_accuracy: 0.0360
Epoch 119/500
46/46 [==============================] - 1s 13ms/step - loss: 0.1104 - accuracy: 0.9688 - val_loss: 50.7132 - val_accuracy: 0.0360
Epoch 120/500
46/46 [==============================] - 1s 16ms/step - loss: 0.1650 - accuracy: 0.9701 - val_loss: 50.4981 - val_accuracy: 0.0

46/46 [==============================] - 1s 15ms/step - loss: 0.0857 - accuracy: 0.9787 - val_loss: 56.6985 - val_accuracy: 0.0370
Epoch 171/500
46/46 [==============================] - 1s 15ms/step - loss: 0.0692 - accuracy: 0.9804 - val_loss: 57.2497 - val_accuracy: 0.0360
Epoch 172/500
46/46 [==============================] - 1s 14ms/step - loss: 0.0979 - accuracy: 0.9761 - val_loss: 54.9421 - val_accuracy: 0.0370
Epoch 173/500
46/46 [==============================] - 1s 15ms/step - loss: 0.0816 - accuracy: 0.9780 - val_loss: 54.6026 - val_accuracy: 0.0370
Epoch 174/500
46/46 [==============================] - 1s 15ms/step - loss: 0.1387 - accuracy: 0.9699 - val_loss: 56.5579 - val_accuracy: 0.0370
Epoch 175/500
46/46 [==============================] - 1s 15ms/step - loss: 0.0967 - accuracy: 0.9780 - val_loss: 55.2996 - val_accuracy: 0.0370
Epoch 176/500
46/46 [==============================] - 1s 15ms/step - loss: 0.0884 - accuracy: 0.9803 - val_loss: 57.6863 - val_accuracy: 0.0370

46/46 [==============================] - 1s 15ms/step - loss: 0.0804 - accuracy: 0.9815 - val_loss: 64.8387 - val_accuracy: 0.0370
Epoch 227/500
46/46 [==============================] - ETA: 0s - loss: 0.0677 - accuracy: 0.98 - 1s 15ms/step - loss: 0.0688 - accuracy: 0.9808 - val_loss: 62.7314 - val_accuracy: 0.0370
Epoch 228/500
46/46 [==============================] - 1s 15ms/step - loss: 0.0653 - accuracy: 0.9808 - val_loss: 62.2697 - val_accuracy: 0.0370
Epoch 229/500
46/46 [==============================] - 1s 14ms/step - loss: 0.1013 - accuracy: 0.9800 - val_loss: 60.8785 - val_accuracy: 0.0370
Epoch 230/500
46/46 [==============================] - 1s 15ms/step - loss: 0.0794 - accuracy: 0.9810 - val_loss: 63.0314 - val_accuracy: 0.0370
Epoch 231/500
46/46 [==============================] - 1s 15ms/step - loss: 0.0935 - accuracy: 0.9766 - val_loss: 61.4972 - val_accuracy: 0.0370
Epoch 232/500
46/46 [==============================] - 1s 15ms/step - loss: 0.0748 - accuracy: 0.9746 

...

Epoch 274/500
46/46 [==============================] - 0s 11ms/step - loss: 0.0916 - accuracy: 0.9821 - val_loss: 69.5853 - val_accuracy: 0.0370
Epoch 275/500
46/46 [==============================] - 0s 11ms/step - loss: 0.1045 - accuracy: 0.9785 - val_loss: 66.1762 - val_accuracy: 0.0370
Epoch 276/500
46/46 [==============================] - 0s 9ms/step - loss: 0.0863 - accuracy: 0.9808 - val_loss: 66.9982 - val_accuracy: 0.0370
Epoch 277/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0793 - accuracy: 0.9818 - val_loss: 67.0028 - val_accuracy: 0.0370
Epoch 278/500
46/46 [==============================] - 0s 9ms/step - loss: 0.1023 - accuracy: 0.9792 - val_loss: 67.7928 - val_accuracy: 0.0370
Epoch 279/500
46/46 [==============================] - 0s 9ms/step - loss: 0.0699 - accuracy: 0.9829 - val_loss: 67.1958 - val_accuracy: 0.0370
Epoch 280/500
46/46 [==============================] - 0s 10ms/step - loss: 0.1022 - accuracy: 0.9829 - val_loss: 68.0611 - val_accur

46/46 [==============================] - 0s 10ms/step - loss: 0.0616 - accuracy: 0.9872 - val_loss: 73.8231 - val_accuracy: 0.0370
Epoch 331/500
46/46 [==============================] - 0s 9ms/step - loss: 0.0553 - accuracy: 0.9830 - val_loss: 71.9549 - val_accuracy: 0.0370
Epoch 332/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0971 - accuracy: 0.9818 - val_loss: 71.8746 - val_accuracy: 0.0370
Epoch 333/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0900 - accuracy: 0.9821 - val_loss: 73.3245 - val_accuracy: 0.0370
Epoch 334/500
46/46 [==============================] - 0s 9ms/step - loss: 0.0985 - accuracy: 0.9757 - val_loss: 72.1669 - val_accuracy: 0.0370
Epoch 335/500
46/46 [==============================] - 0s 9ms/step - loss: 0.1219 - accuracy: 0.9813 - val_loss: 73.8567 - val_accuracy: 0.0370
Epoch 336/500
46/46 [==============================] - 0s 9ms/step - loss: 0.3436 - accuracy: 0.9813 - val_loss: 72.1136 - val_accuracy: 0.0370
Epo

Epoch 387/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0714 - accuracy: 0.9843 - val_loss: 78.9381 - val_accuracy: 0.0370
Epoch 388/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0703 - accuracy: 0.9856 - val_loss: 78.1681 - val_accuracy: 0.0360
Epoch 389/500
46/46 [==============================] - 0s 9ms/step - loss: 0.0821 - accuracy: 0.9827 - val_loss: 79.3588 - val_accuracy: 0.0370
Epoch 390/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0519 - accuracy: 0.9859 - val_loss: 84.0138 - val_accuracy: 0.0370
Epoch 391/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0852 - accuracy: 0.9834 - val_loss: 81.8762 - val_accuracy: 0.0370
Epoch 392/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0941 - accuracy: 0.9784 - val_loss: 82.7122 - val_accuracy: 0.0370
Epoch 393/500
46/46 [==============================] - 0s 10ms/step - loss: 0.1608 - accuracy: 0.9830 - val_loss: 82.7752 - val_acc

46/46 [==============================] - 0s 9ms/step - loss: 0.0869 - accuracy: 0.9881 - val_loss: 85.8875 - val_accuracy: 0.0370
Epoch 444/500
46/46 [==============================] - 0s 9ms/step - loss: 0.0966 - accuracy: 0.9849 - val_loss: 84.9353 - val_accuracy: 0.0350
Epoch 445/500
46/46 [==============================] - 0s 9ms/step - loss: 0.0571 - accuracy: 0.9873 - val_loss: 84.2723 - val_accuracy: 0.0370
Epoch 446/500
46/46 [==============================] - 0s 10ms/step - loss: 0.1052 - accuracy: 0.9779 - val_loss: 83.2429 - val_accuracy: 0.0370
Epoch 447/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0766 - accuracy: 0.9810 - val_loss: 82.6922 - val_accuracy: 0.0370
Epoch 448/500
46/46 [==============================] - 0s 11ms/step - loss: 0.2213 - accuracy: 0.9838 - val_loss: 85.8407 - val_accuracy: 0.0370
Epoch 449/500
46/46 [==============================] - 0s 10ms/step - loss: 0.0714 - accuracy: 0.9865 - val_loss: 83.5724 - val_accuracy: 0.0370
Ep

Epoch 500/500
46/46 [==============================] - 0s 10ms/step - loss: 0.1060 - accuracy: 0.9799 - val_loss: 85.9804 - val_accuracy: 0.0370
accuracy: 75.40%


In [7]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

75.87% (+/- 0.72%)


In [8]:
X_test = test_df[['peak_1', 'h_1', 'k_1', 'l_1', 'peak_2', 'h_2', 'k_2', 'l_2', 'peak_3', 'h_3', 'k_3', 'l_3', 'peak_4', 'h_4', 'k_4', 'l_4', 'peak_5', 'h_5', 'k_5', 'l_5', 'peak_6', 'h_6', 'k_6', 'l_6']]
y_test = test_df['space_code']

In [9]:
measured_test_labels = to_categorical(y_test)

In [10]:
model.evaluate(X_test, measured_test_labels)

25/25 [==============================] - 0s 1ms/step - loss: 86.2559 - accuracy: 0.4280


[86.255859375, 0.4280155599117279]